<a href="https://colab.research.google.com/github/Vartika-bhatt/demo1/blob/main/stable_diffusion_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!pip install git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg] -U einops omegaconf decord xformers==0.0.20 safetensors
!git clone -b dev https://github.com/camenduru/Text-To-Video-Finetuning
!git clone https://huggingface.co/cerspense/zeroscope_v2_dark_30x448x256

/content
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-t_3a59_f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-t_3a59_f
  Resolved https://github.com/huggingface/diffusers to commit cd7071e7506275a1c7c52df9e1a696feafa3b118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.8 

Cloning into 'Text-To-Video-Finetuning'...
remote: Enumerating objects: 973, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 973 (delta 250), reused 231 (delta 221), pack-reused 651
Receiving objects: 100% (973/973), 1.77 MiB | 3.99 MiB/s, done.
Resolving deltas: 100% (571/571), done.
Cloning into 'zeroscope_v2_dark_30x448x256'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 34 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (34/34), 516.81 KiB | 1.36 MiB/s, done.
Filtering content: 100% (5/5), 7.88 GiB | 94.36 MiB/s, done.


In [ ]:
!pip install -q gradio
# Downloading files from the demo repo
import os
os.mkdir('video')


In [ ]:
from google.colab import files

code= '''
import argparse
import os
import warnings
from pathlib import Path
from uuid import uuid4
from utils.lora import inject_inferable_lora
import torch
from diffusers import DPMSolverMultistepScheduler, TextToVideoSDPipeline
from models.unet_3d_condition import UNet3DConditionModel
from einops import rearrange
from torch.nn.functional import interpolate

from train import export_to_video, handle_memory_attention, load_primary_models
from utils.lama import inpaint_watermark


def initialize_pipeline(model, device="cuda", xformers=False, sdp=False):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        scheduler, tokenizer, text_encoder, vae, _unet = load_primary_models(model)
        del _unet #This is a no op
        unet = UNet3DConditionModel.from_pretrained(model, subfolder='unet')
        # unet.disable_gradient_checkpointing()

    pipeline = TextToVideoSDPipeline.from_pretrained(
        pretrained_model_name_or_path=model,
        scheduler=scheduler,
        tokenizer=tokenizer,
        text_encoder=text_encoder.to(device=device, dtype=torch.half),
        vae=vae.to(device=device, dtype=torch.half),
        unet=unet.to(device=device, dtype=torch.half),
    )
    pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
    unet._set_gradient_checkpointing(value=False)
    handle_memory_attention(xformers, sdp, unet)
    vae.enable_slicing()
    return pipeline


def vid2vid(
    pipeline, init_video, init_weight, prompt, negative_prompt, height, width, num_inference_steps, guidance_scale
):
    num_frames = init_video.shape[2]
    init_video = rearrange(init_video, "b c f h w -> (b f) c h w")
    latents = pipeline.vae.encode(init_video).latent_dist.sample()
    latents = rearrange(latents, "(b f) c h w -> b c f h w", f=num_frames)
    latents = pipeline.scheduler.add_noise(
        original_samples=latents * 0.18215,
        noise=torch.randn_like(latents),
        timesteps=(torch.ones(latents.shape[0]) * pipeline.scheduler.num_train_timesteps * (1 - init_weight)).long(),
    )
    if latents.shape[0] != len(prompt):
        latents = latents.repeat(len(prompt), 1, 1, 1, 1)

    do_classifier_free_guidance = guidance_scale > 1.0

    prompt_embeds = pipeline._encode_prompt(
        prompt=prompt,
        negative_prompt=negative_prompt,
        device=latents.device,
        num_images_per_prompt=1,
        do_classifier_free_guidance=do_classifier_free_guidance,
    )

    pipeline.scheduler.set_timesteps(num_inference_steps, device=latents.device)
    timesteps = pipeline.scheduler.timesteps
    timesteps = timesteps[round(init_weight * len(timesteps)) :]

    with pipeline.progress_bar(total=len(timesteps)) as progress_bar:
        for t in timesteps:
            # expand the latents if we are doing classifier free guidance
            latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
            latent_model_input = pipeline.scheduler.scale_model_input(latent_model_input, t)

            # predict the noise residual
            noise_pred = pipeline.unet(latent_model_input, t, encoder_hidden_states=prompt_embeds).sample

            # perform guidance
            if do_classifier_free_guidance:
                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

            # reshape latents
            bsz, channel, frames, width, height = latents.shape
            latents = latents.permute(0, 2, 1, 3, 4).reshape(bsz * frames, channel, width, height)
            noise_pred = noise_pred.permute(0, 2, 1, 3, 4).reshape(bsz * frames, channel, width, height)

            # compute the previous noisy sample x_t -> x_t-1
            latents = pipeline.scheduler.step(noise_pred, t, latents).prev_sample

            # reshape latents back
            latents = latents[None, :].reshape(bsz, frames, channel, width, height).permute(0, 2, 1, 3, 4)

            progress_bar.update()

    video_tensor = pipeline.decode_latents(latents)

    return video_tensor


@torch.inference_mode()
def inference(
    model,
    prompt,
    negative_prompt=None,
    batch_size=1,
    num_frames=16,
    width=256,
    height=256,
    num_steps=50,
    guidance_scale=9,
    init_video=None,
    init_weight=0.5,
    device="cuda",
    xformers=False,
    sdp=False,
    lora_path='',
    lora_rank=64
):
    with torch.autocast(device, dtype=torch.half):
        pipeline = initialize_pipeline(model, device, xformers, sdp)
        inject_inferable_lora(pipeline, lora_path, r=lora_rank)
        prompt = [prompt] * batch_size
        negative_prompt = ([negative_prompt] * batch_size) if negative_prompt is not None else None

        if init_video is not None:
            videos = vid2vid(
                pipeline=pipeline,
                init_video=init_video.to(device=device, dtype=torch.half),
                init_weight=init_weight,
                prompt=prompt,
                negative_prompt=negative_prompt,
                height=height,
                width=width,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
            )

        else:
            videos = pipeline(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_frames=num_frames,
                height=height,
                width=width,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
                output_type="pt",
            ).frames

        return videos


if __name__ == "__main__":
    import decord

    decord.bridge.set_bridge("torch")

    parser = argparse.ArgumentParser()
    parser.add_argument("-m", "--model", type=str, required=True)
    parser.add_argument("-p", "--prompt", type=str, required=True)
    parser.add_argument("-n", "--negative-prompt", type=str, default=None)
    parser.add_argument("-o", "--output-dir", type=str, default="./output")
    parser.add_argument("-B", "--batch-size", type=int, default=1)
    parser.add_argument("-T", "--num-frames", type=int, default=16)
    parser.add_argument("-W", "--width", type=int, default=256)
    parser.add_argument("-H", "--height", type=int, default=256)
    parser.add_argument("-s", "--num-steps", type=int, default=50)
    parser.add_argument("-g", "--guidance-scale", type=float, default=9)
    parser.add_argument("-i", "--init-video", type=str, default=None)
    parser.add_argument("-iw", "--init-weight", type=float, default=0.5)
    parser.add_argument("-f", "--fps", type=int, default=8)
    parser.add_argument("-d", "--device", type=str, default="cuda")
    parser.add_argument("-x", "--xformers", action="store_true")
    parser.add_argument("-S", "--sdp", action="store_true")
    parser.add_argument("-lP", "--lora_path", type=str, default="")
    parser.add_argument("-lR", "--lora_rank", type=int, default=64)
    parser.add_argument("-rw", "--remove-watermark", action="store_true")
    args = vars(parser.parse_args())

    output_dir = args.pop("output_dir")
    prompt = args.get("prompt")
    fps = args.pop("fps")
    remove_watermark = args.pop("remove_watermark")
    init_video = args.pop("init_video")

    if init_video is not None:
        vr = decord.VideoReader(init_video)
        init = rearrange(vr[:], "f h w c -> c f h w").div(127.5).sub(1).unsqueeze(0)
        init = interpolate(init, size=(args["num_frames"], args["height"], args["width"]), mode="trilinear")
        args["init_video"] = init

    videos = inference(**args)

    os.makedirs(output_dir, exist_ok=True)
    out_stem = f"{output_dir}/"
    if init_video is not None:
        out_stem += f"({Path(init_video).stem}) * {args['init_weight']} | "
        path=out_stem
    out_stem += f"{prompt}"

    for video in videos:

        if remove_watermark:
            video = rearrange(video, "c f h w -> f c h w").add(1).div(2)
            video = inpaint_watermark(video)
            video = rearrange(video, "f c h w -> f h w c").clamp(0, 1).mul(255)

        else:
            video = rearrange(video, "c f h w -> f h w c").clamp(-1, 1).add(1).mul(127.5)

        video = video.byte().cpu().numpy()

        export_to_video(video, f"{out_stem}.mp4", fps)
  '''

with open('/content/Text-To-Video-Finetuning/inference.py', 'w') as f:
    f.write(code)



In [ ]:
%cd /content/Text-To-Video-Finetuning
# while True:
prompt = "cars driving on a road" #@param {type:"string"}
negative = "" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 30 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
fps = 24 #@param {type:"raw"}
num_frames = 20 #@param {type:"raw"}
!python inference.py -m "/content/zeroscope_v2_dark_30x448x256" -p {prompt} -n {negative} -W 448 -H 256 -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames}

/content/Text-To-Video-Finetuning
2023-08-11 02:19:31.667073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading pipeline components...: 100% 5/5 [00:00<00:00, 9267.13it/s]
100% 30/30 [00:17<00:00,  1.71it/s]


In [ ]:
prompt=prompt[1:(len(prompt)-1)]

In [ ]:

path="/content/outputs/"+prompt+".mp4"

In [ ]:
display(path)

'/content/outputs/cars driving on a road.mp4'

In [ ]:
import gradio as gr
import os


def video_identity(video):
    return video


demo = gr.Interface(video_identity,
                    gr.Video(),
                    "playable_video",
                    examples=[
                        path],
                    cache_examples=True)

if __name__ == "__main__":
    #print(public_url)
    demo.launch(share=True,debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:332: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4
  warnings.warn(


Caching examples at: '/content/Text-To-Video-Finetuning/gradio_cached_examples/91'
Caching example 1/1
Caching complete

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a604f1449152095520.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a604f1449152095520.gradio.live
